![sql](img/sql-logo.jpg)

# A beginner's guide to databases, SQL, & using them with `pandas`

**Scenario:** You are a data analyst for the Homeland Security, trying to create reports on the active airports world wide. The data you need to access is in a SQL database. YOu need to be able to query for the data in a database!

## Learning goals:
- Goal 1: Summarize the use case for sql in the data science skill set
- Goal 2: Define key sql terminology
- Goal 3: Get information about DB schema and table structure
- Goal 4: Use basic SQL commands:
    - Construct SQL queries
    - Use JOIN to merge tables along logical columns
    - Grouping Data with SQL
- Goal 5: Query data from pandas dataframes using SQL
- Goal 6: Convert SQL to pandas

## Goal 1: Summarize

To date the data we've seen has looked like [this.](https://data.austintexas.gov/Health-and-Community-Services/Austin-Animal-Center-Intakes/wter-evkm)

That is how we need data to look to run analysis and build models.<br>
But it doesn't _live_ there in it's native state.

[Netflix has a great article](https://medium.com/netflix-techblog/notebook-innovation-591ee3221233) describing three different data roles at their company, their different needs, and their toolsets.

![netflix](img/netflix-data-roles.jpeg)

Examining that graphic, SQL shows up as one of the tools of the _Data Engineer_ 

Data Engineers provide the essential data architecture services that make data science possible.

![hierarchy](img/ai-hierachy.png)

[Source: Monica Rogati’s fantastic Medium post “The AI Hierarchy of Needs”
](https://hackernoon.com/the-ai-hierarchy-of-needs-18f111fcc007)

## What does it mean to **Engineer Data**?

Let's start with a basic scenario:<br>
You are HR. You have *no* current database. 
How would you set it up?

Data needs considerations:
- What you want to store
- What "views" you anticipate wanting in the future

Structure considerations:
- Speed of retrieval
- How much data you are accessing
- How much you are storing

![etl](img/etl.png)

[img source: Jeff Hammerbacher’s slide from UC Berkeley CS 194 course ](https://bcourses.berkeley.edu/courses/1377158/pages/cs-194-16-introduction-to-data-science-fall-2015)

### What is a Relational Database? 

![rdb](img/relational-dbms-model.png)
[reference for image ](https://www.studytonight.com/dbms/database-model.php)

### POPULAR RDBMS

- SQLite
- MySQL
- PostgreSql
- Oracle DB
- SQL Server

***
### SQL

[History of SQL](https://www.businessnewsdaily.com/5804-what-is-sql.html)

> Future users of large data banks must be protected from having to know how the data is organized in the machine (the internal representation). A prompting service which supplies such information is not a satisfactory solution. Activities of users at terminals and most application programs should remain unaffected when the internal representation of data is changed and even when some aspects of the external representation are changed. Changes in data representation will often be needed as a result of changes in query, update, and report traffic and natural growth in the types of stored information.
.
.
.<br>
<br>
In Section 1 a relational model of data is proposed as a basis for protecting users of formatted data systems from the potentially disruptive changes in data representation caused by growth in the data bank and changes in traffic.

[The original SQL paper from the 1970s.](https://www.seas.upenn.edu/~zives/03f/cis550/codd.pdf)

***
## Goal 2: Database terminology

### Relational Database Schema

![schema](img/MySQL_Schema_Music_Example.png)

[source of image](https://database.guide/what-is-a-database-schema/)

### Table columns view
<img src="img/columns.png" width=650>

### Terminology

- Database
- Schema
- Queries
- Tables
- Views
- Primary Keys
- Foreign Keys
- Indexes
- Functions & User Defined Functions (UDFs)

***
### SQLite

<img src="img/SQLite-Python.jpg" width=650>

"SQLite is a C library that provides a lightweight disk-based database that doesn’t require a separate server process and allows accessing the database using a nonstandard variant of the SQL query language. Some applications can use SQLite for internal data storage. It’s also possible to prototype an application using SQLite and then port the code to a larger database such as PostgreSQL or Oracle." - [sqlite documentation](https://docs.python.org/2/library/sqlite3.html)



## But what about connecting to database servers?

To connect to other database types there are many different libraries:
- Oracle: [cx_Oracle](https://oracle.github.io/python-cx_Oracle/)
- MySQL: [MySQL-python](https://github.com/farcepest/MySQLdb1)
- PostgreSQL: [Psycopg2](http://initd.org/psycopg/docs/)
- Microsoft SQL Server: [pymssql](http://www.pymssql.org/en/stable/)

These all implement [PEP 249: DB API v2](https://www.python.org/dev/peps/pep-0249/)

***
## Goal 3: Get going with sqlite!

In [32]:
import sqlite3

#### Load a DB object with `connect` and `cursor`

In [2]:
con = sqlite3.connect('data/flights.db')
cursor = con.cursor()

#### Use sqlite_master to find all the tables in the schema
Get the schema of a database from a db in sqlite

In [33]:
cursor.execute("""SELECT name, type FROM sqlite_master WHERE type="table";""")

In [34]:
print(cursor.fetchmany(1))

[('airports', 'table')]


In [9]:
cursor.description

(('name', None, None, None, None, None, None),
 ('type', None, None, None, None, None, None))

In [37]:
cursor.execute("SELECT name, type FROM sqlite_master;")
print(*cursor.fetchall(), sep='\n')

# results = cursor.fetchall()
# print(results[0], results[1], ... results[-1])

('airports', 'table')
('ix_airports_index', 'index')
('airlines', 'table')
('ix_airlines_index', 'index')
('routes', 'table')
('ix_routes_index', 'index')


In [38]:
cursor.execute("SELECT * FROM sqlite_master;")
results = cursor.fetchall()
for res in results:
    print(res)

('table', 'airports', 'airports', 2, 'CREATE TABLE airports (\n[index] INTEGER,\n  [id] TEXT,\n  [name] TEXT,\n  [city] TEXT,\n  [country] TEXT,\n  [code] TEXT,\n  [icao] TEXT,\n  [latitude] TEXT,\n  [longitude] TEXT,\n  [altitude] TEXT,\n  [offset] TEXT,\n  [dst] TEXT,\n  [timezone] TEXT\n)')
('index', 'ix_airports_index', 'airports', 3, 'CREATE INDEX ix_airports_index ON airports ([index])')
('table', 'airlines', 'airlines', 945, 'CREATE TABLE airlines (\n[index] INTEGER,\n  [id] TEXT,\n  [name] TEXT,\n  [alias] TEXT,\n  [iata] TEXT,\n  [icao] TEXT,\n  [callsign] TEXT,\n  [country] TEXT,\n  [active] TEXT\n)')
('index', 'ix_airlines_index', 'airlines', 946, 'CREATE INDEX ix_airlines_index ON airlines ([index])')
('table', 'routes', 'routes', 1393, 'CREATE TABLE routes (\n[index] INTEGER,\n  [airline] TEXT,\n  [airline_id] TEXT,\n  [source] TEXT,\n  [source_id] TEXT,\n  [dest] TEXT,\n  [dest_id] TEXT,\n  [codeshare] TEXT,\n  [stops] TEXT,\n  [equipment] TEXT\n)')
('index', 'ix_routes_i

#### Get information about one table

**A note about** `execute`<br>
Each time you use it, you reset the value of cursor

In [39]:
cursor.execute("SELECT * FROM airports as a").fetchone()

(0,
 '1',
 'Goroka',
 'Goroka',
 'Papua New Guinea',
 'GKA',
 'AYGA',
 '-6.081689',
 '145.391881',
 '5282',
 '10',
 'U',
 'Pacific/Port_Moresby')

In [40]:
cursor.fetchone()

(1,
 '2',
 'Madang',
 'Madang',
 'Papua New Guinea',
 'MAG',
 'AYMD',
 '-5.207083',
 '145.7887',
 '20',
 '10',
 'U',
 'Pacific/Port_Moresby')

In [41]:
cursor.fetchmany(4)

[(2,
  '3',
  'Mount Hagen',
  'Mount Hagen',
  'Papua New Guinea',
  'HGU',
  'AYMH',
  '-5.826789',
  '144.295861',
  '5388',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (3,
  '4',
  'Nadzab',
  'Nadzab',
  'Papua New Guinea',
  'LAE',
  'AYNZ',
  '-6.569828',
  '146.726242',
  '239',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (4,
  '5',
  'Port Moresby Jacksons Intl',
  'Port Moresby',
  'Papua New Guinea',
  'POM',
  'AYPY',
  '-9.443383',
  '147.22005',
  '146',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (5,
  '6',
  'Wewak Intl',
  'Wewak',
  'Papua New Guinea',
  'WWK',
  'AYWK',
  '-3.583828',
  '143.669186',
  '19',
  '10',
  'U',
  'Pacific/Port_Moresby')]

#### Use description

In [42]:
cursor.description

(('index', None, None, None, None, None, None),
 ('id', None, None, None, None, None, None),
 ('name', None, None, None, None, None, None),
 ('city', None, None, None, None, None, None),
 ('country', None, None, None, None, None, None),
 ('code', None, None, None, None, None, None),
 ('icao', None, None, None, None, None, None),
 ('latitude', None, None, None, None, None, None),
 ('longitude', None, None, None, None, None, None),
 ('altitude', None, None, None, None, None, None),
 ('offset', None, None, None, None, None, None),
 ('dst', None, None, None, None, None, None),
 ('timezone', None, None, None, None, None, None))

#### Or use `Pragma`
`Pragma` tool [link here](https://www.sqlite.org/pragma.html#pragma_table_info)

**output**<br>
`(column id, column name, data type, whether or not the column can be NULL, and the default value for the column)`

In [43]:
cursor.execute("PRAGMA table_info(airports)")
info = cursor.fetchall()
print(*info, sep = "\n") 

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'id', 'TEXT', 0, None, 0)
(2, 'name', 'TEXT', 0, None, 0)
(3, 'city', 'TEXT', 0, None, 0)
(4, 'country', 'TEXT', 0, None, 0)
(5, 'code', 'TEXT', 0, None, 0)
(6, 'icao', 'TEXT', 0, None, 0)
(7, 'latitude', 'TEXT', 0, None, 0)
(8, 'longitude', 'TEXT', 0, None, 0)
(9, 'altitude', 'TEXT', 0, None, 0)
(10, 'offset', 'TEXT', 0, None, 0)
(11, 'dst', 'TEXT', 0, None, 0)
(12, 'timezone', 'TEXT', 0, None, 0)


In [44]:
cursor.description

(('cid', None, None, None, None, None, None),
 ('name', None, None, None, None, None, None),
 ('type', None, None, None, None, None, None),
 ('notnull', None, None, None, None, None, None),
 ('dflt_value', None, None, None, None, None, None),
 ('pk', None, None, None, None, None, None))

#### Making fetch happen

`.fetchall()` is how you get the query results out of the object.

You can also `.fetchone()` or `.fetchmany()`

**Task:** Get the descriptive data for airlines and routes tables

In [45]:
cursor.execute("PRAGMA table_info(airlines)")
info = cursor.fetchall()
print(*info, sep = "\n")

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'id', 'TEXT', 0, None, 0)
(2, 'name', 'TEXT', 0, None, 0)
(3, 'alias', 'TEXT', 0, None, 0)
(4, 'iata', 'TEXT', 0, None, 0)
(5, 'icao', 'TEXT', 0, None, 0)
(6, 'callsign', 'TEXT', 0, None, 0)
(7, 'country', 'TEXT', 0, None, 0)
(8, 'active', 'TEXT', 0, None, 0)


In [46]:
cursor.execute("PRAGMA table_info(routes)")
info = cursor.fetchall()
print(*info, sep = "\n")

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'airline', 'TEXT', 0, None, 0)
(2, 'airline_id', 'TEXT', 0, None, 0)
(3, 'source', 'TEXT', 0, None, 0)
(4, 'source_id', 'TEXT', 0, None, 0)
(5, 'dest', 'TEXT', 0, None, 0)
(6, 'dest_id', 'TEXT', 0, None, 0)
(7, 'codeshare', 'TEXT', 0, None, 0)
(8, 'stops', 'TEXT', 0, None, 0)
(9, 'equipment', 'TEXT', 0, None, 0)


***
## Goal 4: Use basic SQL commands 
- Construct SQL queries
- Use JOIN to merge tables along logical columns
- Grouping Data with SQL

### Construct SQL queries

**Select**

**From**

**Where**

**Order by**

**Limit**

#### Options for each:

**Select**:  `distinct`, using `as` to rename columns, single number aggregates

**From:** also uses aliasing with `as`

**Where**: equals is only one `=`, `between`, `in`, wildcards `%`, `AND`, `OR`, `NOT`

**Order by**: `asc` and `desc`

**Limit**:  #

**Task**: 
- Select only active airlines in the UK from the airlines table
- Select the unique list of countries with airports

In [47]:
cursor.execute("""
SELECT name
FROM airlines
WHERE active = 'Y'
  AND country = 'United Kingdom';
""")
cursor.fetchmany(5)

[('Astraeus',),
 ('Air Southwest',),
 ('Aurigny Air Services',),
 ('Air Wales',),
 ('AD Aviation',)]

In [8]:
statement = """
-- Select the unique list of countries with airports
SELECT DISTINCT country
FROM airports
WHERE country LIKE 'Gu%'
"""

cursor.execute(statement)
print(*cursor.fetchall(), sep='\n')

('Guernsey',)
('Guinea-Bissau',)
('Guinea',)
('Guatemala',)
('Guam',)
('Guyana',)
('Guadeloupe',)


### SQL Joins

SQL joins can be used to both **add** data to a table and **remove** data from a table. 

<img src="img/venn.png" width=550>

**Task** Write a query that will join the latitude and longitude data from the airports table to the information on the routes table

In [ ]:
cursor.execute("""
SELECT DISTINCT r.source, dest, latitude, longitude
FROM airports a
JOIN routes r
--  ON a.id = r.source_id
""")
print(*cursor.fetchall(), sep='\n')

### Grouping statements

Combines `select` and `group by` when you want aggregates by values

`select` `min(x)` ... `max()`, `sum()`, etc

`group by x`

**Task**<br>
- Which countries have the highest amount of active airlines?
- Which countries have the highest amount of inactive airlines?
- What about airports by timezones?

In [7]:
# Which countries have the highest amount of active airlines?
cursor.execute("""
SELECT
    country
  , count(*) AS airline_count
FROM airlines
WHERE active = 'Y'
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
""")
print(*cursor.fetchall(), sep='\n')

('United States', 141)
('Russia', 72)
('United Kingdom', 40)
('Germany', 37)
('Canada', 34)
('Australia', 26)
('China', 25)
('Spain', 24)
('Brazil', 23)
('France', 22)


In [8]:
cursor.description

(('country', None, None, None, None, None, None),
 ('count(*)', None, None, None, None, None, None))

In [9]:
# Which countries have the highest amount of inactive airlnes?
cursor.execute("""
SELECT
    country
  , count(*) AS airline_count
FROM airlines
WHERE active = 'N'
GROUP BY country
ORDER BY count(*) DESC
LIMIT 10

""")
print(*cursor.fetchall(), sep='\n')

('United States', 939)
('Mexico', 427)
('United Kingdom', 367)
('Canada', 284)
('Russia', 158)
('Spain', 142)
('France', 97)
('Germany', 94)
('South Africa', 81)
('Nigeria', 80)


In [11]:
# What about airports by timezones?
cursor.execute("""
SELECT
    timezone AS tz
  , count(*) AS a_cnt
FROM airports
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
""")
print(*cursor.fetchall(), sep='\n')

('America/New_York', 628)
('America/Chicago', 373)
('Europe/Berlin', 319)
('America/Anchorage', 258)
('Europe/Paris', 232)
('America/Los_Angeles', 226)
('Asia/Chongqing', 222)
('Europe/London', 193)
('America/Toronto', 159)
('Asia/Calcutta', 141)


In [12]:
cursor.description

(('tz', None, None, None, None, None, None),
 ('a_cnt', None, None, None, None, None, None))

In [ ]:
# It's always a good idea to close our connections when we're done
cursor.close()
con.close()

In [13]:
import pandas as pd
pd.read_sql("""SELECT
    timezone AS tz
  , count(*) AS a_cnt
FROM airports
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10""", con)

,tz,a_cnt
0,America/New_York,628
1,America/Chicago,373
2,Europe/Berlin,319
3,America/Anchorage,258
4,Europe/Paris,232
5,America/Los_Angeles,226
6,Asia/Chongqing,222
7,Europe/London,193
8,America/Toronto,159
9,Asia/Calcutta,141


## Goal 5: Using sql within pandas to filter

`.query()`

[query documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html)

In [14]:
# Get data for an example
import pandas as pd
shelter_data = pd.read_csv('https://data.austintexas.gov/api/views/9t4d-g238/rows.csv?accessType=DOWNLOAD')
shelter_data.dtypes

Animal ID           object
Name                object
DateTime            object
MonthYear           object
Date of Birth       object
Outcome Type        object
Outcome Subtype     object
Animal Type         object
Sex upon Outcome    object
Age upon Outcome    object
Breed               object
Color               object
dtype: object

In [20]:
# shelter_data.loc[shelter_data.Name == 'Max']
max_data = shelter_data.query('Name == "Max" and Breed == "Pug Mix"')
max_data.head()
# #vs
# shelter_data[shelter_data['Name']=="Max"]

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
24,A736717,Max,10/21/2016 02:45:00 PM,10/21/2016 02:45:00 PM,10/16/2013,Return to Owner,NaN,Dog,Neutered Male,3 years,Pug Mix,Brown Brindle
66140,A736717,Max,11/27/2016 11:53:00 AM,11/27/2016 11:53:00 AM,10/16/2013,Return to Owner,NaN,Dog,Neutered Male,3 years,Pug Mix,Brown Brindle
67895,A662933,Max,10/12/2013 05:13:00 PM,10/12/2013 05:13:00 PM,09/10/2006,Return to Owner,NaN,Dog,Neutered Male,7 years,Pug Mix,Brown/Black
78173,A736717,Max,06/29/2017 01:41:00 PM,06/29/2017 01:41:00 PM,10/16/2013,Return to Owner,NaN,Dog,Neutered Male,3 years,Pug Mix,Brown Brindle
107567,A734858,Max,07/08/2017 02:42:00 PM,07/08/2017 02:42:00 PM,09/14/2015,Return to Owner,NaN,Dog,Intact Male,1 year,Pug Mix,Brown/Black


In [21]:
shelter_data.rename(index=str, columns={"Animal Type": "animal_type"}, inplace = True)
test = shelter_data.query('animal_type == "Dog"')
test.head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,animal_type,Sex upon Outcome,Age upon Outcome,Breed,Color
0,A720371,Moose,02/13/2016 05:59:00 PM,02/13/2016 05:59:00 PM,10/08/2015,Adoption,NaN,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
10,A740079,Terry,12/26/2016 01:45:00 PM,12/26/2016 01:45:00 PM,12/10/2013,Return to Owner,NaN,Dog,Neutered Male,3 years,Australian Cattle Dog Mix,Black/White
12,A725530,King,06/09/2016 02:50:00 PM,06/09/2016 02:50:00 PM,10/30/2015,Adoption,NaN,Dog,Neutered Male,7 months,Pit Bull Mix,Brown Brindle/White
13,A722378,Canine,03/31/2016 07:08:00 PM,03/31/2016 07:08:00 PM,03/15/2010,Return to Owner,NaN,Dog,Neutered Male,6 years,German Shepherd Mix,Brown/Black
15,A681449,Buster,07/02/2014 07:01:00 PM,07/02/2014 07:01:00 PM,06/16/2010,Return to Owner,NaN,Dog,Neutered Male,4 years,Dogo Argentino Mix,White/Brown


## Goal 6: Transfering from sqlite to pandas

In [ ]:
pd_con = sqlite3.connect("flights.db")
df = pd.read_sql_query("select * from airports limit 5;", pd_con)
df

**Task**: 
Convert one of the earlier queries in the lesson to a pandas data frame

In [ ]:
# Which countries have the highest amount of active airports?
df = pd.read_sql_query("""
-- your code here;
""", pd_con)
df

In [ ]:
pd_con.close()

## Integration

The [Chinook database](https://github.com/lerocha/chinook-database) is a sample database, representing a digital media store.

You need to create a query that can rank tracks in term of popularity.

The name of the database is `Chinook_Sqlite.sqlite`

Database information:<br>
- How many tables are in the database?
- What's the primary key of each table?
- What foreign keys join the tables together?
- If you had to draw a schema of how the tables are connected, what would it look like?

To answer the question:<br>
- What are the max and min dates in the Invoice table?
- What tables would you need to answer "what is your most popular track?"
- What values from each table?

In [34]:
conn = sqlite3.connect('data/Chinook_Sqlite.sqlite')
cur = conn.cursor()

In [24]:
# How many tables are in the database?
pd.read_sql("""SELECT count(*) FROM sqlite_master WHERE type="table";""", conn)
# your code here

,count(*)
0,11


In [29]:
# What's the primary key of each table?
cur.execute("""SELECT name FROM sqlite_master WHERE type='table'""")
results = cur.fetchall()
for row in results:
    table = row[0]
    cur.execute(f"PRAGMA table_info({table})")
    columns = cur.fetchall()
    pk = ''
    for column in columns:
        if column[5] == 1:
            pk = column[1]
    print(f'Table {table} has primary key {pk}')
            
# your code here

Table Album has primary key AlbumId
Table Artist has primary key ArtistId
Table Customer has primary key CustomerId
Table Employee has primary key EmployeeId
Table Genre has primary key GenreId
Table Invoice has primary key InvoiceId
Table InvoiceLine has primary key InvoiceLineId
Table MediaType has primary key MediaTypeId
Table Playlist has primary key PlaylistId
Table PlaylistTrack has primary key PlaylistId
Table Track has primary key TrackId


In [ ]:
# What foreign keys join the tables together?

# your code here
# hint: use "PRAGMA foreign_key_list()"


In [ ]:
# What are the max and min dates in the Invoice table?
cur.execute("""
-- your code here;
""").fetchall()

In [ ]:
# What tables would you need to answer "what is your most popular track?"


In [ ]:
# What values from each table?


In [31]:
# Put it all together:
# You need to create a query that can rank tracks in term of popularity.

results = cur.execute("""
SELECT
  Name,
  sum(Quantity) AS purchase_count
FROM Track t
JOIN InvoiceLine il ON t.TrackId = il.TrackId
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
""").fetchall()

print(*results, sep='\n')

('The Trooper', 5)
('Eruption', 4)
('Hallowed Be Thy Name', 4)
('Sure Know Something', 4)
('The Number Of The Beast', 4)
('Untitled', 4)
('2 Minutes To Midnight', 3)
('Blood Brothers', 3)
('Brasil', 3)
('Can I Play With Madness', 3)


In [36]:
# Advanced: get the artist who sang the song!
results = cur.execute("""
SELECT
  t.Name track_name,
  ar.Name AS artist_name,
  sum(Quantity) AS purchase_count
FROM Track t
JOIN InvoiceLine il USING (TrackId)
JOIN Album al USING (AlbumId)
JOIN Artist ar USING (ArtistId)
GROUP BY 1
ORDER BY 3 DESC
LIMIT 10
""").fetchall()

print(*results, sep='\n')

('The Trooper', 'Iron Maiden', 5)
('Eruption', 'Van Halen', 4)
('Hallowed Be Thy Name', 'Iron Maiden', 4)
('Sure Know Something', 'Kiss', 4)
('The Number Of The Beast', 'Iron Maiden', 4)
('Untitled', 'Pearl Jam', 4)
('2 Minutes To Midnight', 'Iron Maiden', 3)
('Blood Brothers', 'Iron Maiden', 3)
('Brasil', 'Cássia Eller', 3)
('Can I Play With Madness', 'Iron Maiden', 3)


## Reflection

## Learning goals review:
- Goal 1: Summarize the use case for sql in the data science skill set
- Goal 2: Define key sql terminology
- Goal 3: Get information about DB schema and table structure
- Goal 4: Use basic SQL commands:
    - Construct SQL queries
    - Use JOIN to merge tables along logical columns
    - Grouping Data with SQL
- Goal 5: Query data from pandas dataframes using SQL
- Goal 6: Convert SQL to pandas

Please fill out today's [Exit Ticket](https://docs.google.com/forms/d/1O0dMGqjIV6-JIDYSIeGqTjIdep1BFiIVCc29UbEgwUM/edit)!